In [10]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animation


import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff


# Step 1: Loading Dataset

In [11]:
# df = pd.read_csv('Data\FS_Classification_AMZN_Historical_Quarterly_2009_2022_With_Fundamental_Data_Economic_Indicators.csv')
df = pd.read_csv('Data\FS_Classification_AMZN_Historical_Quarterly_2023_Onwards_With_Fundamental_Data_Economic_Indicators.csv')



# Removing leading and trailing spaces from column names
df.columns = df.columns.str.strip()

# Using a regular expression to replace multiple spaces with a single space in all column names
df.columns = df.columns.str.replace(r'\s+', ' ', regex=True)  

# # Dropping columns that are not needed
df.drop(["Date", "Year"], axis=1, inplace=True)



# Step 2: Overview of Dataset

In [12]:
num_of_rows = len(df)
print(f"The number of rows is {num_of_rows}")
print('\n')

df.info()

The number of rows is 7


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 37 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   RSI                                    7 non-null      float64
 1   Stochastic_Oscillator                  7 non-null      float64
 2   Price_Gap                              7 non-null      float64
 3   Federal_Funds_Rate_Quarterly           7 non-null      float64
 4   cashAndCashEquivalentsAtCarryingValue  7 non-null      float64
 5   totalNonCurrentAssets                  7 non-null      float64
 6   intangibleAssets                       7 non-null      float64
 7   shortTermInvestments                   7 non-null      float64
 8   otherCurrentAssets                     7 non-null      float64
 9   shortLongTermDebtTotal                 7 non-null      float64
 10  commonStock                            7 non-null   

In [13]:
df.head()

,RSI,Stochastic_Oscillator,Price_Gap,Federal_Funds_Rate_Quarterly,cashAndCashEquivalentsAtCarryingValue,totalNonCurrentAssets,intangibleAssets,shortTermInvestments,otherCurrentAssets,shortLongTermDebtTotal,...,interestExpense,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,Quarterly_Return,Volatility_Adjusted_Class
0,54.283704,57.035177,-0.064516,4.516667,4.934300e+10,1.374170e+11,2.274900e+10,1.506200e+10,3.770000e+08,7.057200e+10,...,823000000.0,4.120000e+09,9.480000e+08,823000000.0,3.686000e+09,4.943000e+09,6.489000e+09,3.172000e+09,0.262078,4
1,62.979858,74.222630,0.270968,4.990000,4.952900e+10,1.433560e+11,2.278500e+10,1.444100e+10,9.000000e+09,6.857200e+10,...,840000000.0,7.554000e+09,8.040000e+08,840000000.0,7.043000e+09,8.394000e+09,9.933000e+09,6.750000e+09,-0.024854,2
2,53.048983,52.517575,0.140317,5.260000,4.960500e+10,1.474310e+11,2.274900e+10,1.456400e+10,5.320000e+09,6.763800e+10,...,806000000.0,1.218500e+10,2.306000e+09,806000000.0,8.556000e+09,1.299100e+10,1.443000e+10,9.879000e+09,0.195249,4
3,57.185846,69.167466,0.082222,5.330000,7.338700e+10,1.590190e+11,3.047600e+10,1.339300e+10,6.897000e+09,6.732900e+10,...,713000000.0,1.366600e+10,3.042000e+09,713000000.0,1.258700e+10,1.437900e+10,1.646100e+10,1.062400e+10,0.187179,4
4,59.673912,71.302744,0.187542,5.330000,7.285200e+10,1.570360e+11,2.277000e+10,1.222200e+10,5.568000e+09,6.690200e+10,...,644000000.0,1.289800e+10,2.467000e+09,644000000.0,9.873000e+09,1.354200e+10,1.448300e+10,1.043100e+10,0.071349,3


# Step 3: EDA - Missing Values Analysis 

## Step 3)i): EDA - Show Missing Values in each Column

In [14]:
def display_columns_with_null_values(df: pd.DataFrame):
    """
    Displays the total number of null values for each column in the dataframe,
    showing only columns that have null values.
    
    Parameters:
    - df (pd.DataFrame): The dataframe to be checked for null values.
    
    Returns:
    - None: Prints the columns with null values and their counts.
    """
    
    # Get total null values in each column
    total_null_values = df.isnull().sum()
    
    # Filter out columns that don't have any null values
    columns_with_null = total_null_values[total_null_values > 0].sort_values(ascending=False)
    
    # Check if there are any columns with null values
    if not columns_with_null.empty:
        print('-' * 64)
        print("Total null values in each column (only columns with null values)")
        print('-' * 64)
        print(columns_with_null)
    else:
        print('-' * 64)
        print("Total null values in each column (only columns with null values)")
        print('-' * 64)
        print("No columns have null values.")

In [15]:
# Get percentage of null values in each column
null_values_percentage = df.isnull().mean().round(4).mul(100).sort_values(ascending=False)
print('-' * 44)
print("Percentage(%) of null values in each column")
print('-' * 44)
print(null_values_percentage)
print('\n')

# Get total null values in each column
display_columns_with_null_values(df)


--------------------------------------------
Percentage(%) of null values in each column
--------------------------------------------
RSI                                      0.0
proceedsFromRepurchaseOfEquity           0.0
surprise                                 0.0
surprisePercentage                       0.0
operatingIncome                          0.0
sellingGeneralAndAdministrative          0.0
investmentIncomeNet                      0.0
netInterestIncome                        0.0
interestExpense                          0.0
incomeBeforeTax                          0.0
incomeTaxExpense                         0.0
interestAndDebtExpense                   0.0
comprehensiveIncomeNetOfTax              0.0
ebit                                     0.0
ebitda                                   0.0
netIncome_y                              0.0
Quarterly_Return                         0.0
netIncome_x                              0.0
paymentsForRepurchaseOfEquity            0.0
Stochastic_

## Step 3)ii): EDA - Handling Missing Values

In [16]:
# Fill Null Values in the Remaining Columns with the average of the column
numeric_df = df.select_dtypes(include=[np.number]) # Select only numeric columns
numeric_df.fillna(numeric_df.mean(), inplace=True)  # Fill missing values in numeric columns with the column mean
df[numeric_df.columns] = numeric_df # Merge back with non-numeric columns if needed

# Get total null values in each column
display_columns_with_null_values(df)


----------------------------------------------------------------
Total null values in each column (only columns with null values)
----------------------------------------------------------------
No columns have null values.


# Step 4: EDA - Duplicate Values Analysis 

## Step 4)i): EDA - Show Duplicate Values Rows

In [17]:
# Get percentage of duplicate rows
total_rows = len(df)
duplicate_rows = df.duplicated().sum()
duplicate_percentage = (duplicate_rows / total_rows) * 100

print('-' * 48)
print("Percentage(%) of duplicate rows in the DataFrame")
print('-' * 48)
print(f"{duplicate_percentage:.2f}%")
print('\n')

# Get total number of duplicate rows
print('-' * 30)
print("Total number of duplicate rows")
print('-' * 30)
print(duplicate_rows)


------------------------------------------------
Percentage(%) of duplicate rows in the DataFrame
------------------------------------------------
0.00%


------------------------------
Total number of duplicate rows
------------------------------
0


# Step 5: Backtesting

In [18]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

# Load the saved model
joblib_file = "Model\catboost_model_pipeline.joblib"
loaded_model = joblib.load(joblib_file)


# Extract features and actual labels
features = df.drop(columns=['Volatility_Adjusted_Class']) 
actual_labels = df['Volatility_Adjusted_Class']

# Make predictions using the loaded model
predicted_labels = loaded_model.predict(features)

# Add predictions to the original DataFrame as a new column
df['Volatility_Adjusted_Class_Prediction'] = predicted_labels

df.head(20)  

,RSI,Stochastic_Oscillator,Price_Gap,Federal_Funds_Rate_Quarterly,cashAndCashEquivalentsAtCarryingValue,totalNonCurrentAssets,intangibleAssets,shortTermInvestments,otherCurrentAssets,shortLongTermDebtTotal,...,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,Quarterly_Return,Volatility_Adjusted_Class,Volatility_Adjusted_Class_Prediction
0,54.283704,57.035177,-0.064516,4.516667,4.934300e+10,1.374170e+11,2.274900e+10,1.506200e+10,3.770000e+08,7.057200e+10,...,4.120000e+09,9.480000e+08,8.230000e+08,3.686000e+09,4.943000e+09,6.489000e+09,3.172000e+09,0.262078,4,3
1,62.979858,74.222630,0.270968,4.990000,4.952900e+10,1.433560e+11,2.278500e+10,1.444100e+10,9.000000e+09,6.857200e+10,...,7.554000e+09,8.040000e+08,8.400000e+08,7.043000e+09,8.394000e+09,9.933000e+09,6.750000e+09,-0.024854,2,1
2,53.048983,52.517575,0.140317,5.260000,4.960500e+10,1.474310e+11,2.274900e+10,1.456400e+10,5.320000e+09,6.763800e+10,...,1.218500e+10,2.306000e+09,8.060000e+08,8.556000e+09,1.299100e+10,1.443000e+10,9.879000e+09,0.195249,4,3
3,57.185846,69.167466,0.082222,5.330000,7.338700e+10,1.590190e+11,3.047600e+10,1.339300e+10,6.897000e+09,6.732900e+10,...,1.366600e+10,3.042000e+09,7.130000e+08,1.258700e+10,1.437900e+10,1.646100e+10,1.062400e+10,0.187179,4,3
4,59.673912,71.302744,0.187542,5.330000,7.285200e+10,1.570360e+11,2.277000e+10,1.222200e+10,5.568000e+09,6.690200e+10,...,1.289800e+10,2.467000e+09,6.440000e+08,9.873000e+09,1.354200e+10,1.448300e+10,1.043100e+10,0.071349,3,4
5,53.876961,60.945101,0.256349,5.330000,7.117800e+10,1.607980e+11,2.287900e+10,1.791400e+10,5.906000e+09,6.275900e+10,...,1.525200e+10,1.767000e+09,5.890000e+08,1.309000e+10,1.467200e+10,1.562000e+10,1.348500e+10,-0.035809,2,1
6,50.993386,57.869881,0.227343,5.263333,2.069308e+10,6.976287e+10,9.838672e+09,1.257367e+10,2.440475e+09,2.415570e+10,...,2.579597e+09,3.863387e+08,2.559672e+08,2.174295e+09,2.812774e+09,4.384306e+09,2.193258e+09,0.077790,3,4


In [19]:
# Calculate performance metrics for multi-class classification
accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels, average='macro')  # Average for multi-class
recall = recall_score(actual_labels, predicted_labels, average='macro')
conf_matrix = confusion_matrix(actual_labels, predicted_labels)

# Display the results
# print(f'Accuracy: {accuracy:.4f}')
# print(f'Precision (Macro): {precision:.4f}')
# print(f'Recall (Macro): {recall:.4f}')
print('Confusion Matrix:')
print(conf_matrix)
print('\nClassification Report:')
print(classification_report(actual_labels, predicted_labels))


Confusion Matrix:
[[0 0 0 0]
 [2 0 0 0]
 [0 0 0 2]
 [0 0 3 0]]

Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       0.0
           2       0.00      0.00      0.00       2.0
           3       0.00      0.00      0.00       2.0
           4       0.00      0.00      0.00       3.0

    accuracy                           0.00       7.0
   macro avg       0.00      0.00      0.00       7.0
weighted avg       0.00      0.00      0.00       7.0

